In [1]:
import os
from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession
import warnings

warnings.filterwarnings("ignore")

# Encontrar o caminho absoluto do diretório atual
notebook_cwd = os.getcwd()

# Definir o caminho para a raiz do projeto Kedro com base na estrutura de diretórios
# Neste caso, navegamos 4 níveis acima até a raiz do projeto
project_path = os.path.abspath(os.path.join(notebook_cwd, "../../../mtg-project"))

# Verificar o diretório atual e o caminho do projeto
print(f"Notebook current working directory: {notebook_cwd}")
print(f"Project path: {project_path}")

# Alterar para o diretório raiz do projeto Kedro
os.chdir(project_path)

# Bootstrap o projeto Kedro
bootstrap_project(project_path)

# Inicialize o contexto do Kedro
with KedroSession.create() as session:
    context = session.load_context()

[09/14/24 13:34:38] INFO     Using                                                                  ]8;id=487081;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=90629;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py#249\249]8;;\
                             'c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framew                
                             ork\project\rich_logging.yml' as logging configuration.                               

Notebook current working directory: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\notebooks\gmferratti\pipeline
Project path: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project


In [13]:
# Caso seja necessario recarregar, rodar esta celula
from kedro.framework.session import KedroSession

# Recarregar o contexto e o catálogo
with KedroSession.create() as session:
    context = session.load_context()

# Recarregar o catálogo
catalog = context.catalog

catalog.list()

[09/14/24 14:03:38] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=513592;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=751428;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         


[
    'decks_json_partitioned',
    'decks_txt_partitioned',
    'parameters',
    'params:global',
    'params:global.run_date',
    'params:global.run_date.day',
    'params:global.run_date.month',
    'params:global.run_date.year',
    'params:global.user',
    'params:global.user.project_path',
    'params:preprocessing',
    'params:preprocessing.webscraper',
    'params:preprocessing.webscraper.zip_url',
    'params:preprocessing.webscraper.zip_folder',
    'params:preprocessing.webscraper.log_folder',
    'params:preprocessing.webscraper.valid_card_count',
    'params:preprocessing.webscraper.deck_sample_size_ratio',
    'params:modeling',
    'params:modeling.xgboost',
    'params:modeling.xgboost.learning_rate',
    'params:modeling.xgboost.max_depth',
    'params:modeling.xgboost.n_estimators',
    'params:modeling.random_forest',
    'params:modeling.random_forest.n_estimators',
    'params:modeling.random_forest.max_depth',
    'params:modeling.random_forest.random_state',

In [5]:
import logging

def setup_logger(logger_name:str, 
                 log_folder: str = None) -> logging.Logger:
    """
    Configura o logger para salvar os logs em um arquivo ou exibi-los no terminal.
    
    Args:
        log_folder (str, optional): Caminho do arquivo onde os logs serão salvos. 
        Se for None, o logger exibirá as informações no terminal.
    
    Returns:
        logging.Logger: Logger configurado para salvar logs em arquivo ou exibi-los no terminal.
    """
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)

    # Desativa a propagação do logger para o logger raiz
    logger.propagate = False

    # Verifica se já existem handlers para evitar duplicação de logs
    if not logger.hasHandlers():
        if log_folder:
            # Criar um handler para salvar o log em arquivo
            file_handler = logging.FileHandler(log_folder, mode='w', encoding='utf-8')
            file_handler.setLevel(logging.INFO)

            # Criar um formato para o log
            formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
            file_handler.setFormatter(formatter)

            # Adicionar o handler ao logger
            logger.addHandler(file_handler)
        else:
            # Criar um StreamHandler para exibir os logs no terminal
            console_handler = logging.StreamHandler()
            console_handler.setLevel(logging.INFO)

            # Criar um formato para o log no console
            formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
            console_handler.setFormatter(formatter)

            # Adicionar o handler ao logger
            logger.addHandler(console_handler)

    return logger

In [9]:
import os
import requests
import zipfile

def get_deck_zip_from_web(        
        project_path: str,
        zip_url: str,  
        zip_folder: str) -> None:
    """
    Baixa um arquivo ZIP de um URL, o descompacta em uma pasta temporária e salva os arquivos JSON
    no subdiretório 'decks_json' dentro da pasta especificada.

    Args:
        project_path (str): Caminho base do projeto onde o zip_folder será concatenado.
        zip_url (str): URL do arquivo ZIP a ser baixado.
        zip_folder (str): Caminho relativo dentro do project_path onde o arquivo ZIP será salvo.

    Returns:
        None: A função salva os arquivos JSON na pasta especificada e não retorna nada.
    """
    # Configura o logger geral com o nome "get_deck_zip_logger"
    logger = setup_logger("get_deck_zip_logger")

    # Concatenar o caminho completo de zip_folder com project_path (que é a raiz)
    full_output_path = os.path.join(project_path, zip_folder)
    
    # Criar a pasta raiz se ela não existir
    os.makedirs(full_output_path, exist_ok=True)

    # Criar o subdiretório 'decks_json' dentro da pasta especificada
    decks_json_path = os.path.join(full_output_path, 'decks_json')
    os.makedirs(decks_json_path, exist_ok=True)

    # Função para baixar o arquivo zip
    def download_file(url, folder):
        local_filename = os.path.join(folder, url.split("/")[-1])
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(local_filename, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        logger.info(f"Zip dos decklists baixado com sucesso em: {local_filename}")
        return local_filename

    # Baixar o arquivo zip
    zip_file_path = download_file(zip_url, full_output_path)

    # Descompactar o arquivo zip e salvar os arquivos JSON no subdiretório 'decks_json'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            if file_name.endswith('.json'):
                # Definir o caminho de destino do arquivo extraído no subdiretório 'decks_json'
                output_file_path = os.path.join(decks_json_path, file_name)
                
                # Ler e salvar o arquivo JSON diretamente no caminho especificado
                with zip_ref.open(file_name) as json_file, open(output_file_path, "wb") as out_file:
                    out_file.write(json_file.read())
    
    logger.info(f"Arquivos JSON extraídos com sucesso em: {decks_json_path}")

    # Remover o arquivo zip após extração
    os.remove(zip_file_path)
    logger.info(f"Arquivo ZIP removido após extração!")

    # Remover o handler para evitar problemas futuros
    for handler in logger.handlers:
        handler.close()
        logger.removeHandler(handler)


project_path = catalog.load("params:global.user.project_path")
zip_url = catalog.load("params:preprocessing.webscraper.zip_url")
zip_folder = catalog.load("params:preprocessing.webscraper.zip_folder")

get_deck_zip_from_web(project_path,zip_url, zip_folder)

[09/14/24 13:57:35] INFO     Loading data from params:global.user.project_path                  ]8;id=448774;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=307023;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:preprocessing.webscraper.zip_url          ]8;id=202028;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=762046;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:preprocessing.webscraper.zip_folder       ]8;id=940092;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=70215;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\
                             (MemoryDataset)...                                                                    

2024-09-14 13:57:37,624 - INFO - Zip dos decklists baixado com sucesso em: C:/Users/gufer/OneDrive/Documentos/FIAP/Fase_03/mtg-project\data/01_raw/AllDeckFiles.zip
2024-09-14 13:57:41,424 - INFO - Arquivos JSON extraídos com sucesso em: C:/Users/gufer/OneDrive/Documentos/FIAP/Fase_03/mtg-project\data/01_raw/decks_json
2024-09-14 13:57:41,435 - INFO - Arquivo ZIP removido após extração!


In [14]:
def pp_decks_from_json_files(
        decks_json_partitioned: dict, 
        valid_card_count: int, 
        log_folder: str) -> dict:
    """
    Processa todos os decks JSON fornecidos pelo PartitionedDataSet e salva no formato .txt.

    Faz isso desde que contenham pelo menos o número mínimo de cartas definido em valid_card_count (params).
    O log dos decks processados é salvo em um arquivo no log_folder (params).

    Args:
        decks_json_partitioned (dict): Dicionário de decks carregados de arquivos JSON através de PartitionedDataSet.
        valid_card_count (int): Número mínimo de cartas no mainBoard para que o deck seja processado.
        log_folder (str): Caminho do arquivo onde os logs serão salvos.

    Returns:
        dict: Dicionário de decks processados, onde as chaves são os nomes dos arquivos e os valores são as decklists.
    """
    # Chama a função para configurar o logger
    logger = setup_logger("process_all_decks_logger", log_folder)

    # Função para extrair o nome do deck diretamente do JSON
    def get_deck_name(data):
        return data['data'].get('name', 'Unknown Deck')

    # Função para contar o número de cartas no mainBoard
    def count_mainboard_cards(data):
        return sum(card['count'] for card in data['data']['mainBoard'])

    # Função para gerar a decklist formatada
    def generate_decklist(data):
        decklist = []
        deck_name = get_deck_name(data)
        decklist.append("About")
        decklist.append(f"Name {deck_name}")
        decklist.append("\nDeck")

        for card in data['data']['mainBoard']:
            name = card['name']
            count = card['count']
            decklist.append(f"{count} {name}")

        return decklist

    # Dicionário de saída para armazenar as decklists processadas
    processed_decks = {}

    # Percorrer os arquivos JSON no dicionário fornecido pelo PartitionedDataSet
    for file_name, dataset in decks_json_partitioned.items():
        # Carregar os dados chamando o método `load()` do dataset
        data = dataset()

        # Verificar se o deck tem pelo menos valid_card_count no mainboard
        if count_mainboard_cards(data) >= valid_card_count:
            
            # Gerar a decklist
            decklist = generate_decklist(data)

            # Definir o nome do arquivo de saída .txt com base no nome do deck
            deck_name = get_deck_name(data)
            output_file_name = f"{deck_name.replace(' ', '_')}.txt"

            # Salvar no dicionário de decks processados
            processed_decks[output_file_name] = "\n".join(decklist)

            logger.info(f"Decklist {output_file_name} gerada com sucesso!")
        else:
            logger.info(f"Deck {file_name} ignorado (menos de {valid_card_count} cartas no mainBoard).")

    # Remover o handler para evitar problemas futuros
    for handler in logger.handlers:
        handler.close()
        logger.removeHandler(handler)

    return processed_decks

deck_json_partitioned = catalog.load("decks_json_partitioned")
valid_card_count = catalog.load("params:preprocessing.webscraper.valid_card_count")
log_folder = catalog.load("params:preprocessing.webscraper.log_folder")

txt_decks_dict = pp_decks_from_json_files(deck_json_partitioned, valid_card_count, log_folder)
txt_decks_dict

[09/14/24 14:03:45] INFO     Loading data from decks_json_partitioned (PartitionedDataset)...   ]8;id=278920;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=980105;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from params:preprocessing.webscraper.valid_card_count ]8;id=918926;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=435896;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:preprocessing.webscraper.log_folder       ]8;id=335265;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=666856;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\
                             (MemoryDataset)...                                                                    


{
    'Abzan_Siege.txt': 'About\nName Abzan Siege\n\nDeck\n2 Disowned Ancestor\n3 Ainok Bond-Kin\n1 Sungrace Pegasus\n2 Child of Night\n2 Abzan Falconer\n2 Tuskguard Captain\n1 Abzan Battle Priest\n1 High Sentinels of Arashin\n1 Razorfoot Griffin\n2 Salt Road Patrol\n1 Mer-Ek Nightblade\n2 Longshot Squad\n1 Armament Corps\n1 Ivorytusk Fortress\n1 Carnivorous Moss-Beast\n1 Hunt the Weak\n1 Incremental Growth\n2 Kill Shot\n1 Abzan Charm\n1 Dragonscale Boon\n1 Flesh to Dust\n2 Abzan Banner\n1 Suspension Field\n1 Eternal Thirst\n1 Blossoming Sands\n1 Jungle Hollow\n2 Sandsteppe Citadel\n1 Scoured Barrens\n8 Plains\n7 Swamp\n6 Forest',
    'Adaptive_Enchantment.txt': "About\nName Adaptive Enchantment\n\nDeck\n1 Bruna, Light of Alabaster\n1 Eidolon of Blossoms\n1 Hydra Omnivore\n1 Ajani's Chosen\n1 Celestial Archon\n1 Silent Sentinel\n1 Boon Satyr\n1 Herald of the Pantheon\n1 Cold-Eyed Selkie\n1 Daxos of Meletis\n1 Elderwood Scion\n1 Archetype of Imagination\n1 Whitewater Naiads\n1 Aura Gna

In [27]:
from classes.deck import Deck

